In [1]:
# Imports
load('prismatic_envelope.sage')
load('homology.sage')
load('precision.py')

### User-defined input

In [51]:
# The prime p
p=2

# The motivic weight i in F_p(i)^red
i=8

# The power of the uniformizer n
n=3

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

Fprec,total_precision=target_total_precision(p,i,n)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

# The Eisenstein polynomial E
# USER DEFINED
E=A(z+p)

total p-adic precision is 154
Fprec is 24


### The main calculation

In [52]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

#if i-p+1>0:
#    a_syn0,a_syn1,a_nablaN,a_nablaP,a_gens_N0,a_gens_P0,a_gens_N1,a_gens_P1=syntomic_matrices(p,i-p+1,n,E,total_precision,Fprec,debug=False)
#    b_syn0,b_syn1,b_nablaN,b_nablaP,b_gens_N0,b_gens_P0,b_gens_N1,b_gens_P1=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=False)

In [63]:
syn_cplx_lower=SyntomicComplex(p,i-p+1,n,E,total_precision,Fprec,debug=False)
syn_cplx=SyntomicComplex(p,i,n,E,total_precision+20,Fprec+10,debug=False)

<string>:750: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:769: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:786: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:796: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:810: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:829: DeprecationWarning: __getitem__

In [66]:
syn_cplx.compute_mod_p_homology()
syn_cplx_lower.compute_mod_p_homology()

In [67]:
#syn_cplx.print_mod_p_homology()

In [70]:
nablaN_mod_p=Matrix(GF(2),syn_cplx.nablaN)

print(syn_cplx.name_element('N0',23*[0] + [1]))
syn_cplx.name_element('N1',nablaN_mod_p.column(24))

E^0f3+O(F>=34)


'E^0f3∇z+O(F>=34)'

In [73]:
print(nablaN_mod_p)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0

In [74]:
syn0_mod_p = Matrix(GF(2),syn_cplx.syn0)
print(syn0_mod_p)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [76]:
syn0_mod_p.column(18) + syn0_mod_p.column(14)

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0)

In [58]:
print(ascii_art(syn_cplx.complex_mod_p))
print(ascii_art(syn_cplx_lower.complex_mod_p))

           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
           
 0 <-- C_2 

                                                                                               
                                                                                               
                                                                                               
                                                                                               
                                                   

/nix/store/xfj8hwl16qkr5w9gq2jr18pxd1jkd2ch-python3-3.10.13-env/lib/python3.10/site-packages/sage/typeset/character_art.py:397: UserWarning: the console size is smaller than the pretty representation of the object
  warnings.warn("the console size is smaller than the pretty "


# Computing v1

In [10]:
v1N0,v1P0,v1N1,v1P1=v1_matrices(p,i,n,E,total_precision,Fprec,debug=False)

<string>:903: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:912: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:924: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:936: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:948: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:957: DeprecationWarning: __getitem__

In [11]:
v1N0_Fp=Matrix(GF(p),v1N0)
v1P0_Fp=Matrix(GF(p),v1P0)
v1N1_Fp=Matrix(GF(p),v1N1)
v1P1_Fp=Matrix(GF(p),v1P1)

In [12]:
v1N0_Fp

[0 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]
[0 0 0 1 0]

In [13]:
v1P0_Fp

[0 0 0 0 0]
[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]

In [14]:
v1N1_Fp

[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 0]
[0 0 1 0 0]

In [15]:
v1P1_Fp

[0 0 0 0 0]
[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]

In [16]:
v1_0=Matrix(GF(p),v1N0)
v1_1=Matrix(GF(p),block_matrix([[v1P0,0],[0,v1N1]]))
v1_2=Matrix(GF(p),v1P1)

In [17]:
v1_0

[0 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 0]
[0 1 1 0 0]
[0 0 0 1 0]

In [18]:
v1_1

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0]

In [19]:
v1_2

[0 0 0 0 0]
[0 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]

In [20]:
v1_2*syn_cplx_lower.complex_mod_p.differential()[1]==syn_cplx.complex_mod_p.differential()[1]*v1_1

True

In [21]:
v1={0:v1_0,1:v1_1,2:v1_2}
mor = MorphismHomology(syn_cplx_lower.homology_mod_p, syn_cplx.homology_mod_p, v1)

for h in mor.hF.values():
    print(ascii_art(h))
    print('\n')

[0 1]
[1 0]


[1 1]
[1 0]
[0 0]


[]




In [22]:
print("Generators of mod p H^0 (weight {}, Z/{}^{}):\n\n".format(syn_cplx_lower.i, syn_cplx_lower.p, syn_cplx_lower.n))

for gen in syn_cplx_lower.homology_mod_p_generators[0]:
    print(gen + "\n")

Generators of mod p H^0 (weight 1, Z/2^3):


E^1z^2+E^0f0+E^0zf0+O(F>=6)

E^1z+O(F>=6)



In [23]:
h = mor.hF[0]
for i in range(h.ncols()):
    cycle = syn_cplx.homology_mod_p.compute_cycle(0,h.column(i))
    print("v1 * {}\n           = {}".format(syn_cplx_lower.homology_mod_p_generators[0][i], syn_cplx.name_element(0,cycle)))

v1 * E^1z^2+E^0f0+E^0zf0+O(F>=6)
           = E^1z^2f0+O(F>=6)
v1 * E^1z+O(F>=6)
           = E^2z^2+O(F>=6)


In [46]:
def compute_v1(i):
    Fprec,total_precision=target_total_precision(p,i,n)
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
    A.<z>=PowerSeriesRing(W,Fprec)
    E=A(z+p)
    syn_cplx_lower=SyntomicComplex(p,i-p+1,n,E,total_precision,Fprec,debug=False)
    syn_cplx=SyntomicComplex(p,i,n,E,total_precision,Fprec,debug=False)
    syn_cplx.compute_mod_p_homology()
    syn_cplx_lower.compute_mod_p_homology()
    v1N0,v1P0,v1N1,v1P1=v1_matrices(p,i,n,E,total_precision,Fprec,debug=False)
    v1_0=Matrix(GF(p),v1N0)
    v1_1=Matrix(GF(p),block_matrix([[v1P0,0],[0,v1N1]]))
    v1_2=Matrix(GF(p),v1P1)
    v1={0:v1_0,1:v1_1,2:v1_2}
    mor = MorphismHomology(syn_cplx_lower.homology_mod_p, syn_cplx.homology_mod_p, v1)

    print("Generators of mod p H^0 (weight {}, Z/{}^{}):\n\n".format(syn_cplx_lower.i, syn_cplx_lower.p, syn_cplx_lower.n))

    for gen in syn_cplx_lower.homology_mod_p_generators[0]:
        print(gen + "\n")
    print("v1 action:")
    h = mor.hF[0]
    for i in range(h.ncols()):
        cycle = syn_cplx.homology_mod_p.compute_cycle(0,h.column(i))
        print("v1*( {} )\n           = {}".format(syn_cplx_lower.homology_mod_p_generators[0][i], syn_cplx.name_element(0,cycle)))
    print("-------------------------------------\n\n")    

In [48]:
n=2

for i in range(2,10):
    compute_v1(i)

<string>:750: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:769: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:786: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:796: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:810: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See https://github.com/sagemath/sage/issues/14825 for details.
<string>:829: DeprecationWarning: __getitem__

Generators of mod p H^0 (weight 1, Z/2^2):


E^1z+O(F>=4)

v1 action:
v1*( E^1z+O(F>=4) )
           = E^1zf0+O(F>=4)
-------------------------------------


Generators of mod p H^0 (weight 2, Z/2^2):


E^1zf0+O(F>=6)

v1 action:
v1*( E^1zf0+O(F>=6) )
           = 0+O(F>=6)
-------------------------------------


Generators of mod p H^0 (weight 3, Z/2^2):


E^1zf1+E^0f0*f1+O(F>=8)

v1 action:
v1*( E^1zf1+E^0f0*f1+O(F>=8) )
           = 0+O(F>=8)
-------------------------------------


Generators of mod p H^0 (weight 4, Z/2^2):


E^3zf0+E^2f1+E^0f2+O(F>=10)

E^1zf0*f1+O(F>=10)

v1 action:
v1*( E^3zf0+E^2f1+E^0f2+O(F>=10) )
           = E^3zf1+E^1zf2+O(F>=10)
v1*( E^1zf0*f1+O(F>=10) )
           = 0+O(F>=10)
-------------------------------------


Generators of mod p H^0 (weight 5, Z/2^2):


E^3zf1+E^1zf2+O(F>=12)

E^3zf1+E^0f0*f2+O(F>=12)

E^2f0*f1+O(F>=12)

v1 action:
v1*( E^3zf1+E^1zf2+O(F>=12) )
           = E^3zf0*f1+E^1zf0*f2+O(F>=12)
v1*( E^3zf1+E^0f0*f2+O(F>=12) )
           = E^